In [1]:
from datasets import Dataset
import pandas as pd
from table_LLM import *
from natsort import natsorted
from tokenizers import Tokenizer
from tokenizers.models import BPE
from transformers import BertConfig, BertModel, BertTokenizer
import matplotlib.pyplot as plt

2024-09-10 23:36:18.339463: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-10 23:36:18.353715: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-10 23:36:18.357957: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-10 23:36:18.371061: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-10 23:36:19.304945: W tensorflow/compiler/tf2

In [2]:
data = pd.read_csv('/home/hyun/paper/dataset/train_data_1000.csv')
data_x = data.iloc[:, :-1]
data_y = data.iloc[:, -1:]
x_columns = data_x.columns
y_column = data_y.columns

# data["result"] = data["result"].apply(lambda x: f"<num>{str(x)}<num>")

unique_words = table_util.get_unique_word(data, x_columns=x_columns, y_column=y_column)
special_word = ["<EOS>", "<BOS>", "<PAD>", "<UNK>", "<CLS>", "<SEP>", "<MASK>", "is", ",", " ", "<num>"]
number_words = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "."]
unique_words = natsorted(unique_words)


In [3]:
tokenizer = Tokenizer(BPE())
tokenizer.add_tokens(unique_words)
tokenizer.add_special_tokens(special_word)
tokenizer.add_tokens(number_words)

tokenizer.get_vocab()

{'9': 68,
 'Facility5_3': 30,
 'Facility1_2': 1,
 'Facility2_5': 11,
 'Facility1_4': 3,
 'Facility5_5': 32,
 'Facility6_5': 40,
 'Facility2_3': 9,
 'process4': 44,
 '2': 61,
 'Facility1_7': 6,
 'process5': 45,
 'Facility5_4': 31,
 'Facility3_4': 18,
 'Facility3_1': 15,
 ' ': 57,
 'Facility2_2': 8,
 'Facility2_7': 13,
 '8': 67,
 'Facility4_2': 22,
 'process3': 43,
 'Facility6_1': 36,
 'Facility4_5': 25,
 'Facility1_6': 5,
 'Facility4_1': 21,
 ',': 56,
 '3': 62,
 'Facility3_3': 17,
 'Facility6_4': 39,
 'process2': 42,
 '6': 65,
 'process1': 41,
 '<num>': 58,
 'Facility4_7': 27,
 'Facility5_8': 35,
 'Facility3_2': 16,
 'is': 55,
 'Facility2_1': 7,
 'Facility2_6': 12,
 'Facility5_2': 29,
 '<SEP>': 53,
 '0': 59,
 '<EOS>': 48,
 'Facility4_6': 26,
 '1': 60,
 'Facility4_3': 23,
 '7': 66,
 '<PAD>': 50,
 '4': 63,
 'Facility1_3': 2,
 '5': 64,
 'Facility6_3': 38,
 '<BOS>': 49,
 'Facility5_1': 28,
 'Facility5_6': 33,
 '<CLS>': 52,
 'Facility1_1': 0,
 'Facility3_6': 20,
 'Facility5_7': 34,
 'Facilit

In [4]:
encoded_tokens = tokenizer.encode("process4 is Facility4_3, process6 is Facility6_5, process5 is Facility5_3, process1 is Facility1_2, result is 46.13, process3 is Facility3_5<EOS><MASK>")
print(encoded_tokens)
print("ids : ", encoded_tokens.ids)
print("type_ids : ", encoded_tokens.type_ids)
print("tokens : ", encoded_tokens.tokens)
print("offsets : ", encoded_tokens.offsets)
print("attention_mask : ", encoded_tokens.attention_mask)
print("special_tokens_mask : ", encoded_tokens.special_tokens_mask)
print("overflowing : ", encoded_tokens.overflowing)


Encoding(num_tokens=46, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
ids :  [44, 57, 55, 57, 23, 56, 57, 46, 57, 55, 57, 40, 56, 57, 45, 57, 55, 57, 30, 56, 57, 41, 57, 55, 57, 1, 56, 57, 47, 57, 55, 57, 63, 65, 69, 60, 62, 56, 57, 43, 57, 55, 57, 19, 48, 54]
type_ids :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
tokens :  ['process4', ' ', 'is', ' ', 'Facility4_3', ',', ' ', 'process6', ' ', 'is', ' ', 'Facility6_5', ',', ' ', 'process5', ' ', 'is', ' ', 'Facility5_3', ',', ' ', 'process1', ' ', 'is', ' ', 'Facility1_2', ',', ' ', 'result', ' ', 'is', ' ', '4', '6', '.', '1', '3', ',', ' ', 'process3', ' ', 'is', ' ', 'Facility3_5', '<EOS>', '<MASK>']
offsets :  [(0, 8), (8, 9), (9, 11), (11, 12), (12, 23), (23, 24), (24, 25), (25, 33), (33, 34), (34, 36), (36, 37), (37, 48), (48, 49), (49, 50), (50, 58), (58, 59), (59, 61), (61, 62), (62, 7

In [5]:
configuration = BertConfig(
    vocab_size=tokenizer.get_vocab_size(),
    hidden_size=384,
    num_hidden_layers=6,
    num_attention_heads=6,
    intermediate_size=3072,
    hidden_act="gelu",
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    max_position_embeddings=60,
    type_vocab_size=3,
    initializer_range=0.02,
    layer_norm_eps=1e-12,
    position_embedding_type="absolute",
    classifier_dropout=0.1
)
model = BertModel(configuration)


In [6]:
for i in model.named_modules():
    print(i)

('', BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(70, 384, padding_idx=0)
    (position_embeddings): Embedding(60, 384)
    (token_type_embeddings): Embedding(3, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-5): 6 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [ ]:
model.num_parameters()

In [ ]:
model.get_input_embeddings()

In [ ]:
model.set_input_embeddings()

In [ ]:
dir(model)